In [1]:
import hashlib
import random

# =======================
# SM2 椭圆曲线参数 (sm2p256v1)
# =======================
p = int("FFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF00000000FFFFFFFFFFFFFFFF", 16)
a = int("FFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF00000000FFFFFFFFFFFFFFFC", 16)
b = int("28E9FA9E9D9F5E344D5AEF20E93E3FBD6B5F6F4C52C9A7A3EB5C3C4F9DCC73E", 16)
n = int("FFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFF7203DF6B21C6052B53BBF40939D54123", 16)
Gx = int("32C4AE2C1F1981195F9904466A39C9948FE30BBFF2660BE1715A4589334C74C7", 16)
Gy = int("BC3736A2F4F6779C59BDCEE36B692153D0A9877CC62A474002DF32E52139F0A0", 16)
G = (Gx, Gy)

# ========== 椭圆曲线运算 ==========
def inverse_mod(x, m):
    return pow(x, -1, m)

def point_add(P, Q):
    if P == (0, 0): return Q
    if Q == (0, 0): return P
    if P == Q:
        lam = (3 * P[0]*P[0] + a) * inverse_mod(2 * P[1], p) % p
    else:
        lam = (Q[1]-P[1]) * inverse_mod(Q[0]-P[0], p) % p
    x = (lam*lam - P[0] - Q[0]) % p
    y = (lam*(P[0]-x)-P[1]) % p
    return (x, y)

def scalar_mul(k, P):
    R = (0, 0)
    while k > 0:
        if k & 1:
            R = point_add(R, P)
        P = point_add(P, P)
        k >>= 1
    return R

# ========== SM3 哈希 ==========
def sm3_hash(msg):
    return int(hashlib.sha256(msg.encode()).hexdigest(), 16)  # 用SHA256代替

# ========== SM2 签名 & 验签 ==========
def sm2_keygen():
    d = random.randint(1, n-1)
    P = scalar_mul(d, G)
    return d, P

def sm2_sign(msg, d, k=None):
    e = sm3_hash(msg)
    if not k:
        k = random.randint(1, n-1)
    x1, y1 = scalar_mul(k, G)
    r = (e + x1) % n
    s = (inverse_mod(1+d, n) * (k - r*d)) % n
    return (r, s, k)  # 返回k便于测试漏洞

def sm2_verify(msg, sig, P):
    r, s = sig
    e = sm3_hash(msg)
    t = (r + s) % n
    x1, y1 = point_add(scalar_mul(s, G), scalar_mul(t, P))
    R = (e + x1) % n
    return R == r

# =======================
# 场景1：k泄露
# =======================
def attack_leak_k(r, s, k):
    return ((k - s) * inverse_mod(s + r, n)) % n

# =======================
# 场景2：同用户复用k
# =======================
def attack_reuse_k(r1, s1, r2, s2):
    num = (s2 - s1) % n
    den = (s1 - s2 + r1 - r2) % n
    return (num * inverse_mod(den, n)) % n

# =======================
# 场景3：两用户复用k
# =======================
def attack_two_users(k, s2, r2):
    return ((k - s2) * inverse_mod(s2 + r2, n)) % n

# =======================
# 验证PoC
# =======================
print("=== 生成密钥 ===")
dA, PA = sm2_keygen()
dB, PB = sm2_keygen()
print(f"用户A私钥 dA = {dA}")
print(f"用户B私钥 dB = {dB}")

# 测试消息
m1 = "Message1"
m2 = "Message2"

# 签名 (A重用k)
print("\n=== 签名并演示漏洞 ===")
k_fixed = random.randint(1, n-1)
r1, s1, _ = sm2_sign(m1, dA, k_fixed)
r2, s2, _ = sm2_sign(m2, dA, k_fixed)

print(f"A签名1: r1={r1}, s1={s1}")
print(f"A签名2: r2={r2}, s2={s2}")

# ===== 攻击1：已知k恢复dA =====
print("\n[攻击1] 泄露k恢复私钥：")
dA_recovered = attack_leak_k(r1, s1, k_fixed)
print(f"恢复dA = {dA_recovered}, 是否匹配: {dA == dA_recovered}")

# ===== 攻击2：同用户两次签名复用k恢复dA =====
print("\n[攻击2] 两次签名复用k恢复私钥：")
dA_from_2sigs = attack_reuse_k(r1, s1, r2, s2)
print(f"恢复dA = {dA_from_2sigs}, 是否匹配: {dA == dA_from_2sigs}")

# ===== 攻击3：两用户复用相同k =====
print("\n[攻击3] 两个用户复用k恢复对方私钥：")
# B签名用相同k
rB, sB, _ = sm2_sign("OtherMsg", dB, k_fixed)
dB_recovered = attack_two_users(k_fixed, sB, rB)
print(f"恢复dB = {dB_recovered}, 是否匹配: {dB == dB_recovered}")


=== 生成密钥 ===
用户A私钥 dA = 50851933422529271275978076029533441899293849695585445141198444263818486093400
用户B私钥 dB = 15780408721461727533005823696913655609332194835102963746776916474695042431337

=== 签名并演示漏洞 ===
A签名1: r1=57658835617946150441356688540653009370432912733094431162409277191950753766843, s1=56907487134370626209085011634480070376584207945243991418266040769230511410474
A签名2: r2=38349366612414378390989609132900844438270294828514728983931942670903586148468, s2=59097936053022202686110981878472476044593058713430284803967670091958225592690

[攻击1] 泄露k恢复私钥：
恢复dA = 50851933422529271275978076029533441899293849695585445141198444263818486093400, 是否匹配: True

[攻击2] 两次签名复用k恢复私钥：
恢复dA = 50851933422529271275978076029533441899293849695585445141198444263818486093400, 是否匹配: True

[攻击3] 两个用户复用k恢复对方私钥：
恢复dB = 15780408721461727533005823696913655609332194835102963746776916474695042431337, 是否匹配: True
